# LangChain Agent with Qiskit IBM Transpiler MCP Server

This notebook demonstrates how to create an AI agent using LangChain that connects to the **qiskit-ibm-transpiler-mcp-server** via the Model Context Protocol (MCP).

The agent can interact with IBM Quantum's AI-powered transpiler to:
- Perform AI routing on quantum circuits
- Synthesize Clifford circuits (H, S, CX gates)
- Synthesize Linear Function circuits (CX, SWAP gates)
- Synthesize Permutation circuits (SWAP gates)
- Synthesize Pauli Network circuits (H, S, SX, CX, RX, RY, RZ gates)

## Architecture

```
┌─────────────┐     MCP Protocol     ┌───────────────────────────────────┐
│  LangChain  │ ◄──────────────────► │ qiskit-ibm-transpiler-mcp-server  │
│    Agent    │                      │                                   │
└─────────────┘                      │  ┌─────────────────────────────┐  │
                                     │  │   qiskit-ibm-transpiler     │  │
                                     │  └─────────────────────────────┘  │
                                     │               │                   │
                                     └───────────────│───────────────────┘
                                                     ▼
                                            ┌─────────────────┐
                                            │  IBM Quantum    │
                                            │  AI Transpiler  │
                                            └─────────────────┘
```

## Setup

### 1. Install Dependencies

Run these commands in your terminal:

```bash
# Install the MCP server
pip install qiskit-ibm-transpiler-mcp-server

# Install LangChain dependencies
pip install langchain langchain-mcp-adapters python-dotenv

# Install your preferred LLM provider (choose one):
pip install langchain-openai       # For OpenAI
pip install langchain-anthropic    # For Anthropic Claude
pip install langchain-google-genai # For Google Gemini
pip install langchain-ollama       # For local Ollama
pip install langchain-ibm          # For IBM Watsonx
```

### 2. Configure Environment Variables

Set your IBM Quantum token for the AI Transpiler.

You can either:
- Set them in a `.env` file in this directory
- Set them as environment variables
- Enter them in the cell below

In [1]:
import os

from dotenv import load_dotenv

# LangChain imports
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


# Load from .env file if it exists
load_dotenv()

# Or set directly (uncomment and fill in):
# os.environ["QISKIT_IBM_TOKEN"] = "your-ibm-quantum-token"

# Set your LLM provider API key (uncomment the one you're using):
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# Verify configuration
print("Configuration status:")
print(f"  QISKIT_IBM_TOKEN: {'✓ Set' if os.getenv('QISKIT_IBM_TOKEN') else '✗ Not set'}")

Configuration status:
  QISKIT_IBM_TOKEN: ✓ Set


## Choose Your LLM Provider

Run **one** of the following cells based on your preferred LLM provider:

In [ ]:
# Option 1: OpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Using OpenAI GPT-4o")

In [2]:
# Option 2: Anthropic Claude
from langchain_anthropic import ChatAnthropic


llm = ChatAnthropic(model="aws/claude-haiku-4-5", temperature=0)
print("Using Anthropic Claude Sonnet")

Using Anthropic Claude Sonnet


In [ ]:
# Option 3: Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
print("Using Google Gemini Pro")

In [ ]:
# Option 4: Local Ollama (no API key needed)
from langchain_ollama import ChatOllama


llm = ChatOllama(model="llama3.2", temperature=0)
print("Using local Ollama with Llama 3.2")

In [ ]:
# Option 5: IBM Watsonx
from langchain_ibm import ChatWatsonx


llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={"temperature": 0, "max_tokens": 4096},
)
print("Using IBM Watsonx Granite")

## Define the System Prompt

This prompt tells the agent what it can do and how to behave:

In [3]:
SYSTEM_PROMPT = """You are a helpful quantum computing assistant with access to IBM Quantum's AI-powered
transpiler through the MCP server.

You can help users optimize their quantum circuits using:
- AI Routing (ai_routing tool): Insert SWAP operations to make circuits compatible with backend coupling maps
- AI Clifford Synthesis (ai_clifford_synthesis tool): Optimize Clifford circuits (H, S, CX gates)
- AI Linear Function Synthesis (ai_linear_function_synthesis tool): Optimize Linear Function circuits (CX, SWAP gates)
- AI Permutation Synthesis (ai_permutation_synthesis tool): Optimize Permutation circuits (SWAP gates)
- AI Pauli Network Synthesis (ai_pauli_network_synthesis tool): Optimize Pauli Network circuits (H, S, SX, CX, RX, RY, RZ gates)

When optimizing circuits:
1. First use ai_routing to route the circuit for the target backend
2. Then apply the appropriate synthesis pass based on the circuit type
3. Report the optimization improvements (depth reduction, gate count reduction)

The tools accept QASM 3.0 strings as input and return optimized circuits in QPY format.
Always explain the optimization results and improvements achieved.

Available IBM Quantum backends include: ibm_brisbane, ibm_kyiv, ibm_sherbrooke, ibm_fez, etc.
"""

## Define Sample Circuits

Here are some sample QASM circuits for testing:

In [4]:
# Sample Clifford circuit (H, S, CX gates)
SAMPLE_CLIFFORD_CIRCUIT = """OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
h q[0];
cx q[0], q[1];
s q[1];
cx q[1], q[2];
h q[2];
"""

# Sample Linear Function circuit (CX, SWAP gates)
SAMPLE_LINEAR_FUNCTION_CIRCUIT = """OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
cx q[0], q[1];
cx q[1], q[2];
swap q[0], q[2];
cx q[2], q[1];
"""

# Sample Bell state circuit
SAMPLE_BELL_STATE = """OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];
"""

print("Sample circuits defined!")
print("\nClifford circuit:")
print(SAMPLE_CLIFFORD_CIRCUIT)
print("\nLinear Function circuit:")
print(SAMPLE_LINEAR_FUNCTION_CIRCUIT)
print("\nBell state circuit:")
print(SAMPLE_BELL_STATE)

Sample circuits defined!

Clifford circuit:
OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
h q[0];
cx q[0], q[1];
s q[1];
cx q[1], q[2];
h q[2];


Linear Function circuit:
OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
cx q[0], q[1];
cx q[1], q[2];
swap q[0], q[2];
cx q[2], q[1];


Bell state circuit:
OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];



## Create the MCP Client

This configures the connection to the qiskit-ibm-transpiler-mcp-server:

In [5]:
def get_mcp_client():
    """Create and return an MCP client configured for the Qiskit IBM Transpiler server."""
    return MultiServerMCPClient(
        {
            "qiskit-ibm-transpiler": {
                "transport": "stdio",
                "command": "qiskit-ibm-transpiler-mcp-server",
                "args": [],
                "env": {
                    "QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", ""),
                },
            }
        }
    )

## Create the Agent

Now we'll create a function that sets up the agent with a persistent MCP session.

Using a persistent session is important because it:
- Keeps a single MCP server process running
- Reuses the IBM Quantum connection
- Makes tool calls much faster

In [6]:
async def create_agent_with_session(session):
    """Create a LangChain agent using an existing MCP session."""
    # Load tools from the existing session
    tools = await load_mcp_tools(session)
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  - {tool.name}")

    # Create the agent using LangChain's create_agent
    agent = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)
    return agent

## Helper Function to Run Queries

This function sends a query to the agent and returns the response:

In [7]:
async def ask_agent(agent, query: str) -> str:
    """Send a query to the agent and return the response."""
    result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

## Run the Agent

Now let's create the agent and ask it some questions!

The following cell starts the MCP server, creates the agent, and keeps the session open for multiple queries:

In [8]:
# Create MCP client and start a persistent session
mcp_client = get_mcp_client()

print("Starting MCP server and creating agent...")
print("(This may take a few seconds on first run)\n")

Starting MCP server and creating agent...
(This may take a few seconds on first run)



### Example 1: Route a Bell State Circuit

Let's ask the agent to route a Bell state circuit for a specific backend:

In [9]:
async with mcp_client.session("qiskit-ibm-transpiler") as session:
    agent = await create_agent_with_session(session)

    query = f"""Route this Bell state circuit for the ibm_brisbane backend:

{SAMPLE_BELL_STATE}

Tell me about the optimization results."""

    response = await ask_agent(agent, query)
    print(response)

Loaded 6 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - ai_routing_tool
  - ai_linear_function_synthesis_tool
  - ai_clifford_synthesis_tool
  - ai_permutation_synthesis_tool
  - ai_pauli_network_synthesis_tool
Perfect! Here are the optimization results for your Bell state circuit routed to the **IBM Torino** backend:

## Optimization Results

### Original Circuit Metrics:
- **Number of Qubits**: 2
- **Circuit Depth**: 2
- **Circuit Size**: 2 gates
- **Two-Qubit Gates**: 1 (the CX gate)

### Optimized Circuit Metrics:
- **Number of Qubits**: 133 (mapped to physical qubits on IBM Torino)
- **Circuit Depth**: 2
- **Circuit Size**: 2 gates
- **Two-Qubit Gates**: 1

### Improvements:
- **Depth Reduction**: 0 (no change)
- **Two-Qubit Gate Reduction**: 0 (no change)

## Analysis

Your Bell state circuit is already in an optimal form! The routing process successfully mapped your 2-qubit logical circuit to the IBM Torino backend's 133-qubit physical layout. 

**Key Points:**
1.

### Example 2: Optimize a Clifford Circuit

Use AI Clifford synthesis to optimize a Clifford circuit:

In [10]:
async with mcp_client.session("qiskit-ibm-transpiler") as session:
    agent = await create_agent_with_session(session)

    query = f"""First route this Clifford circuit for ibm_brisbane, then apply AI Clifford synthesis:

{SAMPLE_CLIFFORD_CIRCUIT}

Show me the improvements at each step."""

    response = await ask_agent(agent, query)
    print(response)

Loaded 6 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - ai_routing_tool
  - ai_linear_function_synthesis_tool
  - ai_clifford_synthesis_tool
  - ai_permutation_synthesis_tool
  - ai_pauli_network_synthesis_tool
Perfect! Here's a summary of the optimization results:

## Optimization Summary

### Step 1: AI Routing for ibm_fez
**Original Circuit Metrics:**
- Number of qubits: 3
- Circuit depth: 5
- Circuit size: 5 gates
- Two-qubit gates: 2 (CX gates)

**After Routing:**
- Number of qubits: 156 (expanded to include backend connectivity information)
- Circuit depth: 5
- Circuit size: 5 gates
- Two-qubit gates: 2

**Routing Improvements:**
- ✓ Depth reduction: 0 (no change needed)
- ✓ Two-qubit gate reduction: 0 (circuit already compatible)

The routing step confirmed that your circuit is already well-suited for the ibm_fez backend topology. No SWAP operations were needed to make the circuit compatible with the coupling map.

### Step 2: AI Clifford Synthesis
**After Cliffor

### Example 3: Optimize a Linear Function Circuit

Use AI Linear Function synthesis:

In [11]:
async with mcp_client.session("qiskit-ibm-transpiler") as session:
    agent = await create_agent_with_session(session)

    query = f"""Optimize this Linear Function circuit using AI synthesis for ibm_fez backend:

{SAMPLE_LINEAR_FUNCTION_CIRCUIT}"""

    response = await ask_agent(agent, query)
    print(response)

Loaded 6 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - ai_routing_tool
  - ai_linear_function_synthesis_tool
  - ai_clifford_synthesis_tool
  - ai_permutation_synthesis_tool
  - ai_pauli_network_synthesis_tool
Perfect! I've optimized your Linear Function circuit for the **ibm_fez** backend. Here are the results:

## Optimization Summary

**Original Circuit Metrics:**
- Number of qubits: 3
- Circuit depth: 4
- Circuit size: 4 gates
- Two-qubit gates: 4 (3 CX gates + 1 SWAP gate)

**Optimized Circuit Metrics:**
- Number of qubits: 3
- Circuit depth: 4
- Circuit size: 4 gates
- Two-qubit gates: 4

## Analysis

The circuit is already well-optimized for the ibm_fez backend. The AI Linear Function synthesis tool analyzed the circuit structure and determined that:

1. **No improvements were possible** - The current arrangement of CX and SWAP gates is already optimal for this particular linear function circuit
2. **Circuit structure preserved** - The circuit maintains its depth

### Example 4: Ask About the Transpiler

Ask the agent about available optimization options:

In [12]:
async with mcp_client.session("qiskit-ibm-transpiler") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(
        agent, "What AI synthesis passes are available and what types of circuits do they optimize?"
    )
    print(response)

Loaded 6 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - ai_routing_tool
  - ai_linear_function_synthesis_tool
  - ai_clifford_synthesis_tool
  - ai_permutation_synthesis_tool
  - ai_pauli_network_synthesis_tool
# Available AI Synthesis Passes

Here are the AI-powered synthesis passes available through IBM Quantum's transpiler:

## 1. **AI Routing** (`ai_routing_tool`)
- **Purpose**: Routes quantum circuits by inserting SWAP operations for backend compatibility
- **Circuit Types**: General-purpose - works with any circuit
- **Gate Support**: All gate types
- **Key Features**:
  - Respects backend coupling maps
  - Multiple layout modes: keep, improve, optimize
  - Customizable optimization preferences (minimize CNOTs, gates, layers, or noise)
  - Optimization levels 1-3 (speed vs. quality tradeoff)
- **Use Case**: **Use this FIRST** before other synthesis tools to ensure your circuit is compatible with your target backend

## 2. **AI Clifford Synthesis** (`ai_clifford_syn

### Example 5: Interactive Chat

Run this cell to have an interactive conversation with the agent:

In [13]:
async with mcp_client.session("qiskit-ibm-transpiler") as session:
    agent = await create_agent_with_session(session)
    print("Agent ready! Type your questions below.")
    print("Enter 'quit' to stop.\n")

    while True:
        try:
            query = input("You: ").strip()
            if not query:
                continue
            if query.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            response = await ask_agent(agent, query)
            print(f"\nAssistant: {response}\n")
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

Loaded 6 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - ai_routing_tool
  - ai_linear_function_synthesis_tool
  - ai_clifford_synthesis_tool
  - ai_permutation_synthesis_tool
  - ai_pauli_network_synthesis_tool
Agent ready! Type your questions below.
Enter 'quit' to stop.


Assistant: Hello! 👋 I'm your quantum computing assistant, and I'm here to help you optimize quantum circuits using IBM Quantum's AI-powered transpiler!

Here's what I can help you with:

**Circuit Optimization Tools:**
- **AI Routing** - Insert SWAP operations to make your circuits compatible with specific IBM Quantum backends
- **AI Clifford Synthesis** - Optimize circuits with H, S, and CX gates
- **AI Linear Function Synthesis** - Optimize circuits with CX and SWAP gates
- **AI Permutation Synthesis** - Optimize circuits with SWAP gates
- **AI Pauli Network Synthesis** - Optimize circuits with H, S, SX, CX, RX, RY, RZ gates

**How I can help:**
1. You provide a quantum circuit (in QASM 3.0 format) 

## Custom Queries

Use this cell to optimize your own circuits:

In [14]:
# Enter your QASM circuit here:
MY_CIRCUIT = """OPENQASM 3.0;
include "stdgates.inc";
qubit[3] q;
h q[0];
cx q[0], q[1];
cx q[1], q[2];
"""

MY_BACKEND = "ibm_brisbane"

async with mcp_client.session("qiskit-ibm-transpiler") as session:
    agent = await create_agent_with_session(session)

    query = f"""Route and optimize this circuit for {MY_BACKEND}:

{MY_CIRCUIT}

Show me the optimization improvements."""

    response = await ask_agent(agent, query)
    print(response)

Loaded 6 tools from MCP server:
  - setup_ibm_quantum_account_tool
  - ai_routing_tool
  - ai_linear_function_synthesis_tool
  - ai_clifford_synthesis_tool
  - ai_permutation_synthesis_tool
  - ai_pauli_network_synthesis_tool
It appears that `ibm_brisbane` is not currently available. The system has 37 available backends, but `ibm_brisbane` may not be among them at this moment. 

Could you please try one of these alternative backends instead?
- **ibm_torino** - 5 qubits
- **ibm_fez** - 27 qubits
- **ibm_kyiv** - 9 qubits
- **ibm_sherbrooke** - 33 qubits

Or if you'd like, I can attempt to optimize your circuit with a different backend. Which would you prefer?

Your circuit is quite simple and efficient:
- **Original circuit metrics:**
  - Qubits: 3
  - Depth: 3
  - Size: 3 gates
  - Two-qubit gates: 2 (CX gates)

The circuit creates a linear chain of CNOT gates with an initial Hadamard, which is already well-structured for most backends.


## Available Tools

The agent has access to these tools provided by the MCP server:

| Tool | Description |
|------|-------------|
| `setup_ibm_quantum_account_tool` | Configure IBM Quantum account with API token |
| `ai_routing` | Route circuits for backend coupling maps (insert SWAPs) |
| `ai_clifford_synthesis` | Optimize Clifford circuits (H, S, CX gates) |
| `ai_linear_function_synthesis` | Optimize Linear Function circuits (CX, SWAP gates) |
| `ai_permutation_synthesis` | Optimize Permutation circuits (SWAP gates) |
| `ai_pauli_network_synthesis` | Optimize Pauli Network circuits (H, S, SX, CX, RX, RY, RZ gates) |

## Troubleshooting

### Authentication errors?
- Verify your `QISKIT_IBM_TOKEN` is correct
- Get your token from: https://quantum.ibm.com/

### MCP server not found?
Make sure `qiskit-ibm-transpiler-mcp-server` is installed: `pip install qiskit-ibm-transpiler-mcp-server`

### Circuit not optimizing?
- Ensure your QASM circuit is valid QASM 3.0 syntax
- Check that the circuit contains gates supported by the synthesis pass
- Try a different backend (ibm_brisbane, ibm_fez, ibm_sherbrooke)